# Import libs

In [1]:
import os
import sys
import copy
from collections import defaultdict
from tqdm.notebook import tqdm
import random
import numpy as np
import matplotlib.pyplot as plt


import torch
import torch.nn as nn
import torch.nn.functional as F

sys.path.insert(0, os.path.dirname(os.path.abspath("")))
from utils.data_utils import data_utils
from utils.fetch_features import *
from models import symprop_maml_basenet, symprop_maml_metalearner

random.seed(35)
plt.style.use("seaborn")
%matplotlib inline

print("torch.version:  ", torch.__version__)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("current_device: ", device)
if torch.cuda.is_available():
    print("device_name:    ", torch.cuda.get_device_name(device))


torch.version:   1.9.0+cu111
current_device:  cuda:0
device_name:     GeForce RTX 3090


# Generate Train Dataset & Test Dataset

In [14]:
# -----------------------------
# read free explore motion primitive data
# -----------------------------

file_path = os.path.join(os.path.dirname(os.getcwd()), "archive", "processed", "cap_65d.pickle")
cap_dict = data_utils.read_from_pickle(file_path)

DATA_FEATURE_NUM = len(cap_dict[list(cap_dict.keys())[0]][0][0])
print("DATA_FEATURE_NUM: ", DATA_FEATURE_NUM)

DATA_FEATURE_NUM:  65


## Compute Mean&Std

In [2]:
def get_minibatch_mean_and_std(data_dict):
    mean = 0
    std = 0
    sample_num = 10
    for i in range(sample_num):
        temp_means = []
        temp_stds = []
        for k in data_dict:
            sample_index = np.random.randint(0, len(data_dict[k]), int(0.6 * len(data_dict[k])))
            for index in sample_index:
                index_k_mean = np.mean(np.array(data_dict[k][index]), axis=0)
                index_k_std = np.std(np.array(data_dict[k][index]), axis=0)
                temp_means.append(index_k_mean)
                temp_stds.append(index_k_std)

        mean = mean + np.mean(np.array(temp_means), axis=0)
        #         print('mean: ', np.mean(np.array(temp_means), axis=0))
        std = std + np.mean(np.array(temp_stds), axis=0)

    return (mean / sample_num, std / sample_num)


# z_score_mean, z_score_std = get_minibatch_mean_and_std(cap_dict)
# np.save("symprop_maml_mean.npy", z_score_mean)
# np.save("symprop_maml_std.npy", z_score_std)

## Compute Dataset

In [3]:
z_score_mean = np.load("symprop_maml_mean.npy")
z_score_std = np.load("symprop_maml_std.npy")


def z_score_normalization(x, mean, std):
    return (x - mean) / (std + 1e-5)


def generate_dateset(input_data_dict, time_indent, time_step, z_score_mean, z_score_std):

    datadict = defaultdict()
    #     ["move", "pick_cube", "transport", "place_cube"]
    for motion in FETCH_CAPABILITY_INDEX:
        motion_end = []
        motion_mid = []
        motion_start = []

        for _1_data in input_data_dict[motion]:

            if len(_1_data) < 3 * time_indent:
                continue

            #             for index in range(0, time_indent, 1):
            #                 if (index + time_step) < (time_indent):
            #                     motion_start.append(
            #                         z_score_normalization(
            #                             np.array(_1_data[index : index + time_step], dtype=float), z_score_mean, z_score_std
            #                         )
            #                     )

            for index in range(time_indent, len(_1_data[:-time_indent]), 5):
                if (index + time_step) < (len(_1_data) - time_indent):
                    motion_mid.append(
                        z_score_normalization(
                            np.array(_1_data[index : index + time_step], dtype=float), z_score_mean, z_score_std
                        )
                    )

            for index in range(len(_1_data) - time_indent, len(_1_data), 1):
                if (index + time_step) < len(_1_data):
                    motion_end.append(
                        z_score_normalization(
                            np.array(_1_data[index : index + time_step], dtype=float), z_score_mean, z_score_std
                        )
                    )

        if len(motion_start) == 0:
            datadict[motion] = (motion_mid, motion_end)
        else:
            datadict[motion] = (motion_start, motion_mid, motion_end)

    # 找到长度最小的数据
    min_len = None
    lengths = []
    for motion in datadict:
        for d in datadict[motion]:
            lengths.append(len(d))

    min_len = min(lengths)

    print("min_len: ", min_len)

    datasets = []

    for motion in FETCH_CAPABILITY_INDEX:
        #         data_2 = []
        data_1 = []
        data_0 = []

        selected_index = np.random.choice(range(len(datadict[motion][0])), min_len, replace=False).tolist()
        for index in selected_index:
            data_0.append(datadict[motion][0][index])

        selected_index = np.random.choice(range(len(datadict[motion][1])), min_len, replace=False).tolist()
        for index in selected_index:
            data_1.append(datadict[motion][1][index])

        #         selected_index = np.random.choice(range(len(datadict[motion][2])), min_len, replace=False).tolist()
        #         for index in selected_index:
        #             data_2.append(datadict[motion][2][index])

        datasets.append(data_0)
        datasets.append(data_1)
    #         datasets.append(data_2)

    datasets = np.array(datasets, dtype=float)

    return datasets


# min_batch_cap_dict = dict((k, v[0:150]) for k, v in zip(cap_dict.keys(), cap_dict.values()))
# cap_train_datasets = generate_dateset(
#     cap_dict, time_indent=150, time_step=50, z_score_mean=z_score_mean, z_score_std=z_score_std
# )
# print("cap_train_datasets.shape: ", cap_train_datasets.shape)
# np.save("cap_train_datasets.npy", cap_train_datasets)



# Prepare Data Generator

In [10]:
cap_train_datasets = np.load("cap_train_datasets.npy")

x_test = cap_train_datasets[:, :20000, :, :]
x_train = cap_train_datasets[:, 20000:, :, :]

# x_train = cap_train_datasets
# x_test = cap_test_datasets
# x_predict = np.array()

print(x_train.shape)
print(x_test.shape)
# print(x_predict.shape)

(8, 71500, 50, 65)
(8, 20000, 50, 65)


In [11]:
n_way = 2  #
k_spt = 8  # support data 的个数
k_query = 8  # query data 的个数

time_step = 50
data_feature_num = 65

task_num = 8
batch_size = task_num

# indexes = {"train": 0, "test": 0, "predict": 0}
# datasets = {"train": x_train, "test": x_test, "predict": x_predict}
# print("DB: train", x_train.shape, "test", x_test.shape, "predict", x_predict.shape)
indexes = {"train": 0, "test": 0}
datasets = {"train": x_train, "test": x_test}


def load_data_cache(dataset):

    if dataset.shape[0] == 0:
        return []

    setsz = k_spt * n_way
    querysz = k_query * n_way
    data_cache = []

    # print('preload next 10 caches of batch_size of batch.')
    # 提前载入10个batch的数据
    for sample in range(10):  # num of epochs

        x_spts, y_spts, x_qrys, y_qrys = [], [], [], []

        for i in range(batch_size):  # one batch means one set

            x_spt, y_spt, x_qry, y_qry = [], [], [], []

            # 从多类中选择n_way个类
            selected_cls = np.random.choice(dataset.shape[0], n_way, replace=False)

            for j, cur_class in enumerate(selected_cls):

                selected_data = np.random.choice(dataset.shape[1], k_spt + k_query, replace=False)

                # 构造support集和query集
                x_spt.append(dataset[cur_class][selected_data[:k_spt]])
                x_qry.append(dataset[cur_class][selected_data[k_spt:]])
                y_spt.append([j for _ in range(k_spt)])
                y_qry.append([j for _ in range(k_query)])

            # shuffle inside a batch
            perm = np.random.permutation(n_way * k_spt)
            x_spt = np.array(x_spt).reshape(n_way * k_spt, time_step, data_feature_num)[perm]
            y_spt = np.array(y_spt).reshape(n_way * k_spt)[perm]
            perm = np.random.permutation(n_way * k_query)
            x_qry = np.array(x_qry).reshape(n_way * k_query, time_step, data_feature_num)[perm]
            y_qry = np.array(y_qry).reshape(n_way * k_query)[perm]

            x_spts.append(x_spt)
            y_spts.append(y_spt)
            x_qrys.append(x_qry)
            y_qrys.append(y_qry)

        #         print(x_spts[0].shape)
        # [b, setsz = n_way * k_spt, 1, 84, 84]
        x_spts = np.array(x_spts).astype(np.float32).reshape(batch_size, setsz, time_step, data_feature_num)
        y_spts = np.array(y_spts).astype(int).reshape(batch_size, setsz)
        # [b, qrysz = n_way * k_query, 1, 84, 84]
        x_qrys = np.array(x_qrys).astype(np.float32).reshape(batch_size, querysz, time_step, data_feature_num)
        y_qrys = np.array(y_qrys).astype(int).reshape(batch_size, querysz)
        #         print(x_qrys.shape)
        data_cache.append([x_spts, y_spts, x_qrys, y_qrys])

    return data_cache


datasets_cache = {
    "train": load_data_cache(x_train),
    "test": load_data_cache(x_test),
    #     "predict": load_data_cache(x_predict),
}  # current epoch data cached


def next(mode="train"):
    """
    Gets next batch from the dataset with name.
    :param mode: The name of the splitting (one of "train", "val", "test")
    :return:
    """
    # update cache if indexes is larger than len(data_cache)
    if indexes[mode] >= len(datasets_cache[mode]):
        indexes[mode] = 0
        datasets_cache[mode] = load_data_cache(datasets[mode])

    next_batch = datasets_cache[mode][indexes[mode]]
    indexes[mode] += 1

    return next_batch

# Train MAML

In [ ]:
def train_maml(meta, epochs):

    for epoch in tqdm(range(epochs)):
        #         start = time.time()
        x_spt, y_spt, x_qry, y_qry = next("train")

        x_spt, y_spt, x_qry, y_qry = (
            torch.from_numpy(x_spt).to(device),
            torch.from_numpy(y_spt).to(device),
            torch.from_numpy(x_qry).to(device),
            torch.from_numpy(y_qry).to(device),
        )

        #         print('x_spt',x_spt.size())
        #         print('y_spt',y_spt.size())
        #         print('x_qry',x_qry.size())
        #         print('y_qry',y_qry.size())
        meta.train()
        meta.net.train()
        accs, loss = meta(x_spt, y_spt, x_qry, y_qry)
        #         end = time.time()

        if epoch % 100 == 0:
            print("epoch:", epoch)
            print(accs)
            print(loss)
        
        if (epoch + 1) % 10000 == 0:
            torch.save(meta.state_dict(), "maml_{}.pt".format(str(epoch+1)))
        
        # 测试
        if epoch == 0 or (epoch + 1) % 1000 == 0:
            meta.eval()
            meta.net.eval()
            accs = []
            for _ in range(1000 // task_num):
                x_spt, y_spt, x_qry, y_qry = next("test")
                x_spt, y_spt, x_qry, y_qry = (
                    torch.from_numpy(x_spt).to(device),
                    torch.from_numpy(y_spt).to(device),
                    torch.from_numpy(x_qry).to(device),
                    torch.from_numpy(y_qry).to(device),
                )

                for x_spt_one, y_spt_one, x_qry_one, y_qry_one in zip(x_spt, y_spt, x_qry, y_qry):
                    test_acc = meta.finetunning(x_spt_one, y_spt_one, x_qry_one, y_qry_one)
                    # print('test_acc', test_acc)
                    accs.append(test_acc)
            print("在mean process之前：", np.array(accs).shape)
            accs = np.array(accs).mean(axis=0).astype(np.float16)
            print("测试集准确率:", accs)

EPOCHS = 80000
maml_model = symprop_maml_metalearner.MetaLearner().to(device)
train_maml(maml_model, epochs=EPOCHS)

  0%|          | 0/80000 [00:00<?, ?it/s]

/home/fdse/.local/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


epoch: 0
[0.453125  0.640625  0.6875    0.7421875 0.796875  0.7890625 0.8203125
 0.8359375 0.8515625 0.84375   0.859375 ]
[0.97979826 0.6820059  0.5790563  0.5288358  0.48937464 0.4673741
 0.44247812 0.4305682  0.41079938 0.40262663 0.38658807]
在mean process之前： (1000, 11)
测试集准确率: [0.4973 0.659  0.7183 0.752  0.772  0.7896 0.799  0.806  0.8135 0.817
 0.8203]
epoch: 100
[0.7734375 0.875     0.8984375 0.8984375 0.8984375 0.90625   0.90625
 0.90625   0.90625   0.90625   0.90625  ]
[0.70430696 0.25113723 0.23139845 0.20891334 0.2022161  0.19424559
 0.19277456 0.18846308 0.1868296  0.18545668 0.18340746]
epoch: 200
[0.234375 0.9375   1.       1.       1.       1.       1.       1.
 1.       1.       1.      ]
[2.290892   0.23647113 0.13221537 0.1129437  0.10037956 0.09180159
 0.0856304  0.08070188 0.07692371 0.0735143  0.07074861]
epoch: 300
[0.3203125 0.8359375 0.859375  0.8828125 0.8671875 0.8828125 0.90625
 0.8828125 0.8828125 0.890625  0.90625  ]
[1.3701955  0.27781877 0.27880982 0.23559

In [41]:
torch.save(maml_model.state_dict(), "maml.pt")

In [9]:
maml_model = symprop_maml_metalearner.MetaLearner().to(device)
maml_model.load_state_dict(torch.load("maml.pt"))

<All keys matched successfully>

# Generate Few-Shot Datasets

In [4]:
# ------------------------------------------
# read segmented trajectory data
# ------------------------------------------
data_path = os.path.join(os.path.dirname(os.getcwd()), "archive", "processed", "traj_stationary_65d.pickle")
traj_list = data_utils.read_from_pickle(data_path)

DATA_FEATURE_NUM = len(traj_list[0][list(traj_list[0].keys())[0]][0])
print("DATA_FEATURE_NUM: ", DATA_FEATURE_NUM)

DATA_FEATURE_NUM:  65


In [9]:
def list_2_dict(data_list):
    data_dict = {}
    for motion in FETCH_CAPABILITY_INDEX:
        data_dict.setdefault(motion, [])

    for data in data_list:
        for motion in data:
            data_dict[motion].append(data[motion])
    return data_dict


# 四个阶段的动作

N_SHOTS = [1,5,10,20]
# z_score_mean, z_score_std = get_minibatch_mean_and_std(list_2_dict(traj_list))
for n_shot in N_SHOTS:
    selected_traj_index = np.random.choice(range(len(traj_list)), n_shot, replace=False)
    print(selected_traj_index)

    selected_traj_list = []
    for i in selected_traj_index:
        selected_traj_list.append(copy.deepcopy(traj_list[i]))

    selected_traj_dict = list_2_dict(selected_traj_list)
    n_shot_dataset = generate_dateset(
        selected_traj_dict, time_indent=150, time_step=50, z_score_mean=z_score_mean, z_score_std=z_score_std
    )
    print("{}_shot_dataset.shape: {}".format(str(n_shot), n_shot_dataset.shape))
    np.save("{}_shot_dataset.npy".format(str(n_shot)), n_shot_dataset)
    print()
# ["move","pick_cube","transport","place_cube","pick_cube_1","transport_1","place_cube_1"]
# print("FETCH_CAPABILITY_INDEX: ", FETCH_CAPABILITY_INDEX)

range(0, 1)
min_len:  100
1_shot_dataset.shape: (8, 100, 50, 65)

range(1, 6)
min_len:  500
5_shot_dataset.shape: (8, 500, 50, 65)

range(6, 16)
min_len:  1000
10_shot_dataset.shape: (8, 1000, 50, 65)

range(16, 36)
min_len:  2000
20_shot_dataset.shape: (8, 2000, 50, 65)

range(36, 144)
min_len:  10800
108_shot_dataset.shape: (8, 10800, 50, 65)

